In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow  # For displaying images in Colab

In [ ]:
# Replace 'your_image.jpg' with the actual image file name
image = cv2.imread('/content/test img.jpg')
# to show image
cv2_imshow(image)

In [ ]:
height, width, channels = image.shape
print(f"Image Size: {width}x{height}")
print(f"Number of Channels: {channels}")

In [ ]:
total_pixels = height * width
print(f"Total Pixels: {total_pixels}")

In [ ]:
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2_imshow(gray_image)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def display_image(title, image, cmap=None):
    plt.figure(figsize=(6,6))
    plt.imshow(image, cmap=cmap)
    plt.title(title)
    plt.axis("off")
    plt.show()

# Task 1: Image Manipulation
image_path = '/content/test img.jpg'  # Replace with your image path
image = cv2.imread(image_path)

# Check if image is loaded correctly
if image is None:
    print(f"Error: Could not load image at path '{image_path}'. Check if the file exists and the path is correct.")
else:
    # Convert BGR to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    display_image("Original RGB Image", image_rgb)
    cv2.imwrite("image_rgb.jpg", cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR))

    # Extract Image Size
    height, width, channels = image.shape
    print(f"Image Size: {width}x{height}, Channels: {channels}")

    # Calculate Total Pixels
    total_pixels = height * width
    print(f"Total Pixels: {total_pixels}")

    # Convert RGB to Grayscale
    gray_image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2GRAY)
    display_image("Grayscale Image", gray_image, cmap='gray')
    cv2.imwrite("image_gray.jpg", gray_image)

    # Convert to Binary Image
    threshold_value = 127
    _, binary_image = cv2.threshold(gray_image, threshold_value, 255, cv2.THRESH_BINARY)
    display_image("Binary Image", binary_image, cmap='gray')
    cv2.imwrite("image_binary.jpg", binary_image)

    # Count Black Pixels
    black_pixel_count = np.sum(binary_image == 0)
    print(f"Black Pixel Count: {black_pixel_count}")
    print(f"Size of Image: {width}x{height}")

In [ ]:
from skimage import filters
from skimage.filters import prewitt
from skimage.filters import roberts

# Task 2: Edge Detection
# Sobel Operator
sobelx = cv2.Sobel(gray_image, cv2.CV_64F, 1, 0, ksize=3)
sobely = cv2.Sobel(gray_image, cv2.CV_64F, 0, 1, ksize=3)
sobel_combined = cv2.magnitude(sobelx, sobely)
display_image("Sobel Edge Detection", sobel_combined, cmap='gray')

# Prewitt Operator
prewitt_x = prewitt(gray_image)
prewitt_y = prewitt(gray_image, axis=0)
prewitt_combined = np.sqrt(prewitt_x*2 + prewitt_y*2)
display_image("Prewitt Edge Detection", prewitt_combined, cmap='gray')

# Roberts Cross Operator
roberts_edges = roberts(gray_image)
display_image("Roberts Edge Detection", roberts_edges, cmap='gray')

# Canny Edge Detector
canny_edges = cv2.Canny(gray_image, 100, 200)
display_image("Canny Edge Detection", canny_edges, cmap='gray')

# Task 2: Image Segmentation
# Global Thresholding
_, global_thresh = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY)
display_image("Global Thresholding", global_thresh, cmap='gray')

# Adaptive Thresholding
adaptive_thresh = cv2.adaptiveThreshold(gray_image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
display_image("Adaptive Thresholding", adaptive_thresh, cmap='gray')

# Canny for Segmentation
canny_segment = cv2.Canny(gray_image, 100, 200)
display_image("Canny Edge Segmentation", canny_segment, cmap='gray')

# Watershed Algorithm
gray_blurred = cv2.GaussianBlur(gray_image, (5,5), 0)
_, binary_thresh = cv2.threshold(gray_blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Marker Labelling
kernel = np.ones((3,3), np.uint8)
opening = cv2.morphologyEx(binary_thresh, cv2.MORPH_OPEN, kernel, iterations=2)
sure_bg = cv2.dilate(opening, kernel, iterations=3)
dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
_, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg, sure_fg)
_, markers = cv2.connectedComponents(sure_fg)
markers = markers + 1
markers[unknown == 255] = 0
markers = cv2.watershed(image_rgb, markers)
image_rgb[markers == -1] = [255, 0, 0]  # Mark boundaries in red
display_image("Watershed Segmentation", image_rgb)